In [18]:
from faker import Faker
import random
import sqlite3
import pandas as pd

fake = Faker()

# Create a database connection
conn = sqlite3.connect('ecommerce.db')
cursor = conn.cursor()

# Create Customers table
cursor.execute('''CREATE TABLE Customers (
                    customer_id INTEGER PRIMARY KEY,
                    name TEXT,
                    shipping_addr TEXT,
                    contact_number TEXT,
                    current_flag INTEGER DEFAULT 1
                )''')

# Create Orders table
cursor.execute('''CREATE TABLE Orders (
                    order_id INTEGER PRIMARY KEY,
                    customer_id INTEGER,
                    order_date TEXT,
                    total_amount REAL NOT NULL,
                    FOREIGN KEY (customer_id) REFERENCES Customers(customer_id)
                )''')

# Create Order Items table
cursor.execute('''CREATE TABLE OrderItems (
                    item_id INTEGER PRIMARY KEY,
                    order_id INTEGER,
                    product_id INTEGER,
                    quantity INTEGER,
                    price REAL,
                    FOREIGN KEY (order_id) REFERENCES Orders(order_id),
                    FOREIGN KEY (product_id) REFERENCES Products(product_id)
                )''')

# Create Products table
cursor.execute('''CREATE TABLE Products (
                    product_id INTEGER PRIMARY KEY,
                    name TEXT,
                    description TEXT
                )''')

cursor.execute('''CREATE TABLE Variants (
                    variant_id INTEGER PRIMARY KEY,
                    name TEXT,
                    parent_product_id INTEGER
                )''') 

# Generate customers
for _ in range(10):
    name = fake.name()
    shipping_addr = fake.address().replace("\n", ", ")
    contact_number = fake.phone_number()
    cursor.execute('''INSERT INTO Customers (name, shipping_addr, contact_number)
                      VALUES (?, ?, ?)''', (name, shipping_addr, contact_number))
    conn.commit()

# Generate products
for i in range(10):
    name = ['iPhone', 'iPhone', 'MacBook', 'MacBook', 'Apple Watch', 'AirPods', 'Apple TV', 'iPod', 'Beats', 'iPad']
    description = fake.sentence(nb_words=6)
    cursor.execute('''INSERT INTO Products (name, description)
                      VALUES (?, ?)''', (name[i], description))
    conn.commit()

# Generate variants
for i in range(4):
    name = ['iPhone 13','iphone 14', 'MacBook Air', 'MacBook pro']
    parent_id = [1,2,3,4]
    cursor.execute('''INSERT INTO Variants (name, parent_product_id)
                      VALUES (?, ?)''', (name[i], parent_id[i]))
    conn.commit()

# Generate orders and order items
for _ in range(100):
    customer_id = random.randint(1, 10)
    order_date = fake.date_between(start_date='-2y', end_date='today')
    total_amount = round(random.uniform(10, 1000), 2)
    cursor.execute('''INSERT INTO Orders (customer_id, order_date, total_amount)
                      VALUES (?, ?, ?)''', (customer_id, order_date, total_amount))
    order_id = cursor.lastrowid

    # Generate order items
    num_items = random.randint(1, 5)
    for _ in range(num_items):
        product_id = random.randint(1, 10)
        quantity = random.randint(1, 10)
        price = round(random.uniform(10, 100), 2)
        cursor.execute('''INSERT INTO OrderItems (order_id, product_id, quantity, price)
                          VALUES (?, ?, ?, ?)''', (order_id, product_id, quantity, price))

    conn.commit()

# Close the database connection
conn.close()

In [19]:
# Create a database connection
conn = sqlite3.connect('ecommerce.db')
cursor = conn.cursor()

# Get orders in dataframe
cursor.execute("SELECT * FROM Orders")
orders = cursor.fetchall()
orders_df = pd.DataFrame(orders, columns=[column[0] for column in cursor.description])

# Get Customers in dataframe
cursor.execute("SELECT * FROM Customers")
customers = cursor.fetchall()
customers_df = pd.DataFrame(customers, columns=[column[0] for column in cursor.description])

# Get Products in dataframe
cursor.execute("SELECT * FROM Products")
products = cursor.fetchall()
products_df = pd.DataFrame(products, columns=[column[0] for column in cursor.description])

# Get Variants in dataframe
cursor.execute("SELECT * FROM Variants")
variants = cursor.fetchall()
variants_df = pd.DataFrame(variants, columns=[column[0] for column in cursor.description])

# Get OrderItems in dataframe
cursor.execute("SELECT * FROM OrderItems")
order_items = cursor.fetchall()
order_items_df = pd.DataFrame(order_items, columns=[column[0] for column in cursor.description])

# Close the database connection
conn.close()

In [20]:
orders_df.head()

,order_id,customer_id,order_date,total_amount
0,1,4,2022-10-15,73.96
1,2,7,2022-08-17,46.19
2,3,4,2022-07-11,323.80
3,4,4,2023-01-20,489.55
4,5,8,2022-11-30,441.56


In [21]:
customers_df.head()

,customer_id,name,shipping_addr,contact_number,current_flag
0,1,Sarah Smith,"17810 Jennifer Brook Apt. 886, West Michael, A...",484-541-3416,1
1,2,Jennifer Wilson,"7174 Suzanne Gardens, East Matthewborough, IN ...",717-671-4030x131,1
2,3,Thomas Petersen,"169 Mcknight Neck, Hughesside, ME 61562",+1-328-786-3307x78570,1
3,4,Evelyn Rivera,"Unit 6732 Box 8135, DPO AA 15240",998.536.9031,1
4,5,Nathan Townsend,"6683 Daniel Route, Hernandezport, VI 38879",+1-714-662-0705x3982,1


In [22]:
products_df.head()

,product_id,name,description
0,1,iPhone,Step course this might yeah attack mention.
1,2,iPhone,Wall other serious structure history.
2,3,MacBook,Suggest case cultural adult.
3,4,MacBook,Never method baby miss.
4,5,Apple Watch,Factor item raise as.


In [23]:
variants_df.head()

,variant_id,name,parent_product_id
0,1,iPhone 13,1
1,2,iphone 14,2
2,3,MacBook Air,3
3,4,MacBook pro,4


In [24]:
order_items_df.head()

,item_id,order_id,product_id,quantity,price
0,1,1,1,1,32.98
1,2,1,4,3,19.18
2,3,1,6,9,53.99
3,4,2,8,3,85.64
4,5,2,10,9,79.84
